In [1]:
import torch 
import torch.nn as nn
from utils.tensor_generator import tensor_gen, one_hot_gen

/home/prabin_linux/ai/pytorch_phase1/venv_torch/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


USING SIGMOID IN PYTORCH

In [2]:
ip_tensor = torch.tensor([[6]])
sigmoid = nn.Sigmoid()
output = sigmoid(ip_tensor)
print(output)

tensor([[0.9975]])


USE SOFTMAX IN PYTORCH 

In [3]:
input_tensor = torch.tensor([[-5.5, -2,1.1,3,0,5.5]])
probabilities = nn.Softmax(dim = -1)
output_tensor = probabilities(input_tensor)
print(output_tensor)


tensor([[1.5197e-05, 5.0325e-04, 1.1171e-02, 7.4689e-02, 3.7186e-03, 9.0990e-01]])


FORWARD PASS

RESTART KERNEL : AFTER EACH TIME WE MAKE CHANGES IN THE utils.tensor_generator.tensor_gen()  

gpt: 
 Additional caveats
If you do from utils import tensor_gen, then reload utils, your imported symbol tensor_gen won’t update unless you re-import it again.

For complex projects, automatic reloaders like %autoreload in Jupyter (IPython) or tools like watchdog exist.

In production code or big projects, use build/restart scripts or live-reload frameworks.

In [4]:
#BINARY CLASSIFICATION FORWARD PASS
input_bc = tensor_gen((5,6))    #(BATCH_SIZE, FEATURES)

#create a binary classification nn

binary_classifier = nn.Sequential(
    nn.Linear(6,4),
    nn.Linear(4,1),
    nn.Sigmoid()
)

# PyTorch’s linear layers are batch-aware. SO ENTIRE BATCH IS PROCESSED SIMULTANEOUSLY 

In [5]:
binary_classifier(input_bc)

tensor([[0.5409],
        [0.5758],
        [0.5453],
        [0.5362],
        [0.5496]], grad_fn=<SigmoidBackward0>)

THIS OUTPUT IS NOT BE MEANINGFUL UNTIL WE UPDATE THE WEIGHTS AND BIASES I.E. TRAIN THE MODEL USING BACK PROPAGATIION

In [11]:
# MULTI-CLASS CLASSIFICATION : FORWARD PASS 
n_classes = 3
mc_input_tensor = tensor_gen((10,6))

mc_classifier = nn.Sequential(
    nn.Linear(6,4),
    nn.Linear(4,n_classes),
    # nn.Softmax()
)

In [12]:
mc_output = mc_classifier(mc_input_tensor)
print(mc_output.shape)
print(mc_output)

torch.Size([10, 3])
tensor([[-0.0245, -0.2355,  0.2224],
        [ 0.0959, -0.3584,  0.3160],
        [ 0.0403, -0.2224,  0.3395],
        [ 0.1978, -0.3101,  0.4453],
        [ 0.0952, -0.2455,  0.3927],
        [ 0.1199, -0.3498,  0.3317],
        [-0.0785, -0.1890,  0.1912],
        [-0.1020, -0.1691,  0.1773],
        [ 0.0413, -0.2889,  0.3080],
        [-0.0353, -0.1486,  0.2941]], grad_fn=<AddmmBackward0>)


REGRESSION IN PYTORCH USING SEQUENTIAL MODULE 

EG: PREDICT THE WEIGHT OF THE ANIMALS USING THEIR FEATURES 

- DON'T USE ACTIVATION FUNCTION AT LAST AND LEAVE THE LOGITS AS IT IS WHICH CAN BE INTERPRETED AS THE (continuous value prediction by the model )

In [8]:
reg_model = nn.Sequential(
    nn.Linear(6,4),
    nn.Linear(4,1)
)

pred_weight = reg_model(mc_input_tensor)
print(pred_weight)
print(pred_weight.shape)

tensor([[ 0.0495],
        [ 0.0383],
        [ 0.1031],
        [ 0.0186],
        [ 0.1674],
        [ 0.2515],
        [ 0.0340],
        [ 0.1080],
        [ 0.1769],
        [-0.0157]], grad_fn=<AddmmBackward0>)
torch.Size([10, 1])


USING LOSS FUNCTIONS TO ASSESS MODEL PREDICTIONS



In [10]:
#ONE HOT ENCODING Y

import torch.nn.functional as F 
from torch.nn import CrossEntropyLoss

print(F.one_hot(torch.tensor(2), num_classes= 3))

tensor([0, 0, 1])


In [19]:
scores = mc_output
one_hot_target = one_hot_gen(batch_size=10, num_classes= 3)
final_indices = one_hot_target.argmax(dim = 1) 
criterion = CrossEntropyLoss()

op = criterion(scores.double(), final_indices.long())
# op = criterion(scores.double(), one_hot_target.double())
print(op)
# print(final_indices)

tensor(1.1667, dtype=torch.float64, grad_fn=<NllLossBackward0>)


CROSS ENTROPY LOSS EXPECTS : CLASS LABELS AND NOT ONE HOT ENCODED ?!

gemini: PyTorch nn.CrossEntropyLoss: This function is designed for efficiency and expects the target to be a class index, not a one-hot vector. For the given example, the correct target input would be a single integer 0, representing the index of the correct class.

In [26]:
'''PyTorch takes class_label indexes and not one hot encoded for loss computation
HERE: pytorch is interpreting it as soft labels i.e target as probabilities'''

scores = torch.tensor([-5.2,4.6, 0.8])
# one_hot_target = torch.tensor([0.8,0.15,0.5])
one_hot_target = torch.tensor([1,0,0])

criterion = CrossEntropyLoss()
# print(criterion(scores.double(), one_hot_target)) #RuntimeError: Expected floating point type for target with class probabilities, got Long
print(criterion(scores.double(), one_hot_target.double()))

tensor(9.8222, dtype=torch.float64)
